In [1]:
#Train:optimize its parameters on our data

#Training process: in each epoch the model makes a guess
# about the out put and calculate the loss
# collects the error with derivative of error with
# respect to its parameters, and then optimzes these
# parameter using gradient descent.

In [2]:
# Prerequiste Code
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
#data
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)
#network
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [3]:
#Hyperparameters
#adjustable parameters 
# that let you control the model optimization process

# how much to update models parameters at each batch/epoch
learning_rate = 1e-3
#the number of data samples propagated through the network
batch_size = 64
#the number times to iterate over the dataset
epochs = 5

In [4]:
#Optimization loop

#In training loop, optimization happens in three steps
# 1.Call optimizer.zero_grad() to reset the gradients of model parameters. 
#   zero parameters at each iteration
# 2.Backpropagate the prediction loss with a call to loss.backwards()
# 3.Once we have our gradients, we call optimizer.step() 
#   to adjust the parameters by the gradients collected in the backward pass.

#Each epoch consists of two main epoch
#Train Loop - iterate over the training dataset 
# and try to converge to optimal parameters.

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()#initialize
        loss.backward()#backward to calculate gradient
        optimizer.step()#step to apply new value of parameters

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

#Validation/Test Loop - iterate over 
# the test dataset to check if model performance is improving.
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():#test, avoid grad calculate,stop tracking computations
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [5]:
#loss function:measure degree of dissimilarity of 
# obtained result to the target value
#nn.MSELoss mean square error used for regression task
#nn.NLLLoss negative log likelihood used for classification
#nn.CrossEntropyLoss conbined nn.LogSoftmax and nn.NLLLoss

#Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

#optimizer: adjusting model parameters 
# to reduce model error in each training step.
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!") 

Epoch 1
-------------------------------
loss: 2.313123  [    0/60000]
loss: 2.293383  [ 6400/60000]
loss: 2.281965  [12800/60000]
loss: 2.269229  [19200/60000]
loss: 2.261389  [25600/60000]
loss: 2.238307  [32000/60000]
loss: 2.232664  [38400/60000]
loss: 2.209471  [44800/60000]
loss: 2.216081  [51200/60000]
loss: 2.174140  [57600/60000]
Test Error: 
 Accuracy: 53.6%, Avg loss: 2.171590 

Epoch 2
-------------------------------
loss: 2.183460  [    0/60000]
loss: 2.168296  [ 6400/60000]
loss: 2.122429  [12800/60000]
loss: 2.127548  [19200/60000]
loss: 2.088964  [25600/60000]
loss: 2.033483  [32000/60000]
loss: 2.048763  [38400/60000]
loss: 1.984143  [44800/60000]
loss: 1.994377  [51200/60000]
loss: 1.902487  [57600/60000]
Test Error: 
 Accuracy: 60.5%, Avg loss: 1.915478 

Epoch 3
-------------------------------
loss: 1.950676  [    0/60000]
loss: 1.915777  [ 6400/60000]
loss: 1.812300  [12800/60000]
loss: 1.833247  [19200/60000]
loss: 1.737293  [25600/60000]
loss: 1.688343  [32000/600